In [ ]:
import requests
import pandas as pd
import time
from datetime import datetime, timedelta

def fetch_binance_hourly(symbol, years, filename):
    interval = "1h"
    limit = 1000
    total_hours = years * 365 * 24

    all_data = []
    end_time = int(time.time() * 1000)

    print(f"\n⏳ Downloading {years} years of hourly data for {symbol}...")

    while True:
        url = f"https://api.binance.com/api/v3/klines?symbol={symbol}&interval={interval}&limit={limit}&endTime={end_time}"

        data = requests.get(url).json()

        # If response is empty or error
        if not data or len(data) == 0 or isinstance(data, dict):
            print("⚠ No more data available — stopping.")
            break

        all_data.extend(data)

        # Move backwards safely
        end_time = data[0][0] - 1

        print(f"✔ Fetched: {len(all_data)} rows so far...")
        time.sleep(0.2)

        # Stop after required period
        if len(all_data) >= total_hours:
            break

    df = pd.DataFrame(all_data, columns=[
        "open_time","open","high","low","close","volume",
        "close_time","quote_volume","num_trades",
        "taker_buy_base","taker_buy_quote","ignore"
    ])

    df["open_time"] = pd.to_datetime(df["open_time"], unit="ms")
    df.to_csv(filename, index=False)

    print(f"✔ Saved {filename} with {len(df)} rows")

# Download 3 years of data
fetch_binance_hourly("BTCUSDT", 3, "btc_binance_3y_hourly.csv")



⏳ Downloading 3 years of hourly data for BTCUSDT...
⚠ No more data available — stopping.
✔ Saved btc_binance_3y_hourly.csv with 0 rows


In [ ]:
!pip install pandas requests


In [ ]:
import requests
import pandas as pd
from zipfile import ZipFile
from io import BytesIO
import os


In [ ]:
import os
import pandas as pd
from zipfile import ZipFile
import requests
from io import BytesIO

def download_binance_kline(symbol, years, interval="1h"):
    base_url = "https://data.binance.vision/data/spot/monthly/klines"
    os.makedirs(symbol, exist_ok=True)

    all_dfs = []

    for year in years:
        for month in range(1, 13):
            month_str = f"{month:02d}"
            filename = f"{symbol}-{interval}-{year}-{month_str}.zip"
            url = f"{base_url}/{symbol}/{interval}/{filename}"

            print(f"📥 Downloading: {filename}")
            r = requests.get(url)

            # Skip files that don’t exist
            if r.status_code != 200:
                print(f"⚠ File not found, skipping: {filename}")
                continue

            # Extract
            with ZipFile(BytesIO(r.content)) as z:
                csv_name = z.namelist()[0]
                print(f"→ Extracting: {csv_name}")
                z.extract(csv_name, path=symbol)

                df = pd.read_csv(f"{symbol}/{csv_name}", header=None)
                all_dfs.append(df)

    # Merge final dataset
    final = pd.concat(all_dfs)

    # Assign column names
    final.columns = [
        "open_time","open","high","low","close","volume",
        "close_time","quote_volume","trades",
        "taker_buy_base","taker_buy_quote","ignore"
    ]

    # Convert timestamp
    final["open_time"] = pd.to_datetime(final["open_time"], unit="ms")

    # Save merged file
    output_name = f"{symbol}_merged_hourly.csv"
    final.to_csv(output_name, index=False)

    print(f"\n✔ DONE! Saved merged dataset: {output_name}")
    return final


In [ ]:
years = [2022, 2023, 2024]

download_binance_kline("BTCUSDT", years)
download_binance_kline("ETHUSDT", years)
download_binance_kline("BNBUSDT", years)
download_binance_kline("SOLUSDT", years)
download_binance_kline("XRPUSDT", years)


📥 Downloading: BTCUSDT-1h-2022-01.zip
→ Extracting: BTCUSDT-1h-2022-01.csv
📥 Downloading: BTCUSDT-1h-2022-02.zip
→ Extracting: BTCUSDT-1h-2022-02.csv
📥 Downloading: BTCUSDT-1h-2022-03.zip
→ Extracting: BTCUSDT-1h-2022-03.csv
📥 Downloading: BTCUSDT-1h-2022-04.zip
→ Extracting: BTCUSDT-1h-2022-04.csv
📥 Downloading: BTCUSDT-1h-2022-05.zip
→ Extracting: BTCUSDT-1h-2022-05.csv
📥 Downloading: BTCUSDT-1h-2022-06.zip
→ Extracting: BTCUSDT-1h-2022-06.csv
📥 Downloading: BTCUSDT-1h-2022-07.zip
→ Extracting: BTCUSDT-1h-2022-07.csv
📥 Downloading: BTCUSDT-1h-2022-08.zip
→ Extracting: BTCUSDT-1h-2022-08.csv
📥 Downloading: BTCUSDT-1h-2022-09.zip
→ Extracting: BTCUSDT-1h-2022-09.csv
📥 Downloading: BTCUSDT-1h-2022-10.zip
→ Extracting: BTCUSDT-1h-2022-10.csv
📥 Downloading: BTCUSDT-1h-2022-11.zip
→ Extracting: BTCUSDT-1h-2022-11.csv
📥 Downloading: BTCUSDT-1h-2022-12.zip
→ Extracting: BTCUSDT-1h-2022-12.csv
📥 Downloading: BTCUSDT-1h-2023-01.zip
→ Extracting: BTCUSDT-1h-2023-01.csv
📥 Downloading: BTCUSDT-1h

,open_time,open,high,low,close,volume,close_time,quote_volume,trades,taker_buy_base,taker_buy_quote,ignore
0,2022-01-01 00:00:00,0.8298,0.8400,0.8297,0.8395,6156400.0,1640998799999,5.142493e+06,7585,2843910.0,2.375779e+06,0
1,2022-01-01 01:00:00,0.8396,0.8528,0.8320,0.8396,15912826.0,1641002399999,1.341477e+07,20973,7967362.0,6.720973e+06,0
2,2022-01-01 02:00:00,0.8396,0.8445,0.8392,0.8425,4478888.0,1641005999999,3.770657e+06,5777,2657275.0,2.236958e+06,0
3,2022-01-01 03:00:00,0.8426,0.8457,0.8404,0.8410,5230777.0,1641009599999,4.408399e+06,5283,2424379.0,2.043653e+06,0
4,2022-01-01 04:00:00,0.8410,0.8430,0.8346,0.8346,5368735.0,1641013199999,4.499242e+06,6038,2269202.0,1.901354e+06,0
...,...,...,...,...,...,...,...,...,...,...,...,...
739,2024-12-31 19:00:00,2.0877,2.0957,2.0796,2.0918,3406149.0,1735675199999,7.112241e+06,27542,1645673.0,3.437002e+06,0
740,2024-12-31 20:00:00,2.0918,2.1052,2.0907,2.1003,3978927.0,1735678799999,8.351741e+06,23437,1846600.0,3.876458e+06,0
741,2024-12-31 21:00:00,2.1002,2.1050,2.0886,2.0982,3576364.0,1735682399999,7.505775e+06,22742,1459968.0,3.064885e+06,0
742,2024-12-31 22:00:00,2.0982,2.1030,2.0736,2.0788,4042114.0,1735685999999,8.448551e+06,21185,1740482.0,3.640088e+06,0


from matplotlib import pyplot as plt
_df_0['open'].plot(kind='hist', bins=20, title='open')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_1['high'].plot(kind='hist', bins=20, title='high')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_2['low'].plot(kind='hist', bins=20, title='low')
plt.gca().spines[['top', 'right',]].set_visible(False)

...

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  xs = series['open_time']
  ys = series['close']
  
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = _df_11.sort_values('open_time', ascending=True)
_plot_series(df_sorted, '')
sns.despine(fig=fig, ax=ax)
plt.xlabel('open_time')
_ = plt.ylabel('close')

from matplotlib import pyplot as plt
_df_12['open'].plot(kind='line', figsize=(8, 4), title='open')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
_df_13['high'].plot(kind='line', figsize=(8, 4), title='high')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
_df_14['low'].plot(kind='line', figsize=(8, 4), title='low')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
_df_15['close'].plot(kind='line', figsize=(8, 4), title='close')
plt.gca().spines[['top', 'right']].set_visible(False)

In [ ]:
import os

for file in os.listdir():
    if file.endswith("BTCUSDT_merged_hourly.csv"):
        print(file)


BTCUSDT_merged_hourly.csv


In [ ]:
btc = pd.read_csv("BTCUSDT_merged_hourly.csv")
btc.head()

,open_time,open,high,low,close,volume,close_time,quote_volume,trades,taker_buy_base,taker_buy_quote,ignore
0,2022-01-01 00:00:00,46216.93,46731.39,46208.37,46656.13,1503.33095,1640998799999,6.987999e+07,38608,806.06536,3.746216e+07,0
1,2022-01-01 01:00:00,46656.14,46949.99,46574.06,46778.14,943.81539,1641002399999,4.412715e+07,31872,491.18067,2.296327e+07,0
2,2022-01-01 02:00:00,46778.14,46928.94,46721.96,46811.77,485.16860,1641005999999,2.272067e+07,24364,206.35113,9.662607e+06,0
3,2022-01-01 03:00:00,46811.77,46916.63,46760.12,46813.20,562.88971,1641009599999,2.636326e+07,19882,248.28212,1.162938e+07,0
4,2022-01-01 04:00:00,46813.21,46887.33,46591.23,46711.05,861.88389,1641013199999,4.027204e+07,23357,397.53563,1.857540e+07,0


In [ ]:
eth = pd.read_csv("ETHUSDT_merged_hourly.csv")
eth.head()

,open_time,open,high,low,close,volume,close_time,quote_volume,trades,taker_buy_base,taker_buy_quote,ignore
0,2022-01-01 00:00:00,3676.22,3730.00,3676.22,3723.04,9023.3740,1640998799999,3.343005e+07,21702,4552.4363,1.686479e+07,0
1,2022-01-01 01:00:00,3723.04,3748.45,3714.10,3724.89,8997.7569,1641002399999,3.357262e+07,21666,4156.5750,1.550816e+07,0
2,2022-01-01 02:00:00,3724.88,3738.47,3722.23,3728.32,5028.4531,1641005999999,1.875731e+07,13886,2224.3701,8.297764e+06,0
3,2022-01-01 03:00:00,3728.41,3737.00,3721.84,3723.96,3543.1853,1641009599999,1.321103e+07,11475,1757.2116,6.552446e+06,0
4,2022-01-01 04:00:00,3723.96,3728.07,3707.24,3708.21,6117.2820,1641013199999,2.273208e+07,14222,3308.0933,1.229167e+07,0


In [ ]:
bnb = pd.read_csv("BNBUSDT_merged_hourly.csv")
bnb.head()

,open_time,open,high,low,close,volume,close_time,quote_volume,trades,taker_buy_base,taker_buy_quote,ignore
0,2022-01-01 00:00:00,511.5,517.8,511.4,517.3,22537.945,1640998799999,1.160661e+07,13862,12150.617,6.257319e+06,0
1,2022-01-01 01:00:00,517.3,520.0,516.2,517.2,19858.246,1641002399999,1.029233e+07,13935,8829.725,4.576046e+06,0
2,2022-01-01 02:00:00,517.2,519.3,517.1,517.8,10209.673,1641005999999,5.291197e+06,8660,4975.246,2.578477e+06,0
3,2022-01-01 03:00:00,517.8,518.8,517.3,518.1,9973.326,1641009599999,5.167151e+06,9681,5023.600,2.602981e+06,0
4,2022-01-01 04:00:00,518.1,518.5,514.8,515.0,13355.034,1641013199999,6.898842e+06,9958,5938.686,3.067794e+06,0


In [ ]:
sol = pd.read_csv("SOLUSDT_merged_hourly.csv")
bnb.head()

,open_time,open,high,low,close,volume,close_time,quote_volume,trades,taker_buy_base,taker_buy_quote,ignore
0,2022-01-01 00:00:00,511.5,517.8,511.4,517.3,22537.945,1640998799999,1.160661e+07,13862,12150.617,6.257319e+06,0
1,2022-01-01 01:00:00,517.3,520.0,516.2,517.2,19858.246,1641002399999,1.029233e+07,13935,8829.725,4.576046e+06,0
2,2022-01-01 02:00:00,517.2,519.3,517.1,517.8,10209.673,1641005999999,5.291197e+06,8660,4975.246,2.578477e+06,0
3,2022-01-01 03:00:00,517.8,518.8,517.3,518.1,9973.326,1641009599999,5.167151e+06,9681,5023.600,2.602981e+06,0
4,2022-01-01 04:00:00,518.1,518.5,514.8,515.0,13355.034,1641013199999,6.898842e+06,9958,5938.686,3.067794e+06,0


In [ ]:
xrp = pd.read_csv("XRPUSDT_merged_hourly.csv")
bnb.head()

,open_time,open,high,low,close,volume,close_time,quote_volume,trades,taker_buy_base,taker_buy_quote,ignore
0,2022-01-01 00:00:00,511.5,517.8,511.4,517.3,22537.945,1640998799999,1.160661e+07,13862,12150.617,6.257319e+06,0
1,2022-01-01 01:00:00,517.3,520.0,516.2,517.2,19858.246,1641002399999,1.029233e+07,13935,8829.725,4.576046e+06,0
2,2022-01-01 02:00:00,517.2,519.3,517.1,517.8,10209.673,1641005999999,5.291197e+06,8660,4975.246,2.578477e+06,0
3,2022-01-01 03:00:00,517.8,518.8,517.3,518.1,9973.326,1641009599999,5.167151e+06,9681,5023.600,2.602981e+06,0
4,2022-01-01 04:00:00,518.1,518.5,514.8,515.0,13355.034,1641013199999,6.898842e+06,9958,5938.686,3.067794e+06,0


In [ ]:
from google.colab import files
files.download("BTCUSDT_merged_hourly.csv")
files.download("ETHUSDT_merged_hourly.csv")
files.download("BNBUSDT_merged_hourly.csv")
files.download("SOLUSDT_merged_hourly.csv")
files.download("XRPUSDT_merged_hourly.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import pandas as pd

btc = pd.read_csv("/content/BTCUSDT_merged_hourly.csv")
btc.head()


,open_time,open,high,low,close,volume,close_time,quote_volume,trades,taker_buy_base,taker_buy_quote,ignore
0,1/1/2022 0:00,46216.93,46731.39,46208.37,46656.13,1503.33095,1.641000e+12,69879986.39,38608,806.06536,3.746216e+07,0
1,1/1/2022 1:00,46656.14,46949.99,46574.06,46778.14,943.81539,1.641000e+12,44127148.12,31872,491.18067,2.296327e+07,0
2,1/1/2022 2:00,46778.14,46928.94,46721.96,46811.77,485.16860,1.641010e+12,22720670.42,24364,206.35113,9.662607e+06,0
3,1/1/2022 3:00,46811.77,46916.63,46760.12,46813.20,562.88971,1.641010e+12,26363260.46,19882,248.28212,1.162938e+07,0
4,1/1/2022 4:00,46813.21,46887.33,46591.23,46711.05,861.88389,1.641010e+12,40272043.74,23357,397.53563,1.857540e+07,0


In [ ]:
btc["open_time"] = pd.to_datetime(btc["open_time"])
btc.head()


,open_time,open,high,low,close,volume,close_time,quote_volume,trades,taker_buy_base,taker_buy_quote,ignore
0,2022-01-01 00:00:00,46216.93,46731.39,46208.37,46656.13,1503.33095,1.641000e+12,69879986.39,38608,806.06536,3.746216e+07,0
1,2022-01-01 01:00:00,46656.14,46949.99,46574.06,46778.14,943.81539,1.641000e+12,44127148.12,31872,491.18067,2.296327e+07,0
2,2022-01-01 02:00:00,46778.14,46928.94,46721.96,46811.77,485.16860,1.641010e+12,22720670.42,24364,206.35113,9.662607e+06,0
3,2022-01-01 03:00:00,46811.77,46916.63,46760.12,46813.20,562.88971,1.641010e+12,26363260.46,19882,248.28212,1.162938e+07,0
4,2022-01-01 04:00:00,46813.21,46887.33,46591.23,46711.05,861.88389,1.641010e+12,40272043.74,23357,397.53563,1.857540e+07,0


In [ ]:
btc = btc.drop_duplicates(subset=["open_time"])
btc.shape


(26303, 12)

In [ ]:
btc = btc[[
    "open_time",
    "open", "high", "low", "close", "volume",
    "quote_volume", "trades", "taker_buy_base", "taker_buy_quote"
]]
btc.head()

,open_time,open,high,low,close,volume,quote_volume,trades,taker_buy_base,taker_buy_quote
0,2022-01-01 00:00:00,46216.93,46731.39,46208.37,46656.13,1503.33095,69879986.39,38608,806.06536,3.746216e+07
1,2022-01-01 01:00:00,46656.14,46949.99,46574.06,46778.14,943.81539,44127148.12,31872,491.18067,2.296327e+07
2,2022-01-01 02:00:00,46778.14,46928.94,46721.96,46811.77,485.16860,22720670.42,24364,206.35113,9.662607e+06
3,2022-01-01 03:00:00,46811.77,46916.63,46760.12,46813.20,562.88971,26363260.46,19882,248.28212,1.162938e+07
4,2022-01-01 04:00:00,46813.21,46887.33,46591.23,46711.05,861.88389,40272043.74,23357,397.53563,1.857540e+07


In [ ]:
btc[[
    "open","high","low","close","volume",
    "quote_volume","trades","taker_buy_base","taker_buy_quote"
]] = btc[[
    "open","high","low","close","volume",
    "quote_volume","trades","taker_buy_base","taker_buy_quote"
]].astype(float)

btc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26303 entries, 0 to 26302
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   open_time        26303 non-null  datetime64[ns]
 1   open             26303 non-null  float64       
 2   high             26303 non-null  float64       
 3   low              26303 non-null  float64       
 4   close            26303 non-null  float64       
 5   volume           26303 non-null  float64       
 6   quote_volume     26303 non-null  float64       
 7   trades           26303 non-null  float64       
 8   taker_buy_base   26303 non-null  float64       
 9   taker_buy_quote  26303 non-null  float64       
dtypes: datetime64[ns](1), float64(9)
memory usage: 2.0 MB


In [ ]:
btc.isnull().sum()


,0
open_time,0
open,0
high,0
low,0
close,0
volume,0
quote_volume,0
trades,0
taker_buy_base,0
taker_buy_quote,0


In [ ]:
btc = btc.fillna(method="ffill")
btc.isnull().sum()


/tmp/ipython-input-3533820226.py:1: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  btc = btc.fillna(method="ffill")


,0
open_time,0
open,0
high,0
low,0
close,0
volume,0
quote_volume,0
trades,0
taker_buy_base,0
taker_buy_quote,0


In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

btc[[
    "open","high","low","close","volume",
    "quote_volume","trades","taker_buy_base","taker_buy_quote"
]] = scaler.fit_transform(
    btc[[
        "open","high","low","close","volume",
        "quote_volume","trades","taker_buy_base","taker_buy_quote"
    ]]
)

btc.head()


,open_time,open,high,low,close,volume,quote_volume,trades,taker_buy_base,taker_buy_quote
0,2022-01-01 00:00:00,0.330079,0.334418,0.335309,0.334813,0.010957,0.023250,0.026772,0.011748,0.024925
1,2022-01-01 01:00:00,0.334822,0.336779,0.339298,0.336130,0.006879,0.014681,0.022101,0.007159,0.015279
2,2022-01-01 02:00:00,0.336139,0.336551,0.340912,0.336493,0.003536,0.007559,0.016895,0.003008,0.006429
3,2022-01-01 03:00:00,0.336502,0.336419,0.341328,0.336509,0.004102,0.008771,0.013787,0.003619,0.007738
4,2022-01-01 04:00:00,0.336518,0.336102,0.339486,0.335406,0.006282,0.013399,0.016197,0.005794,0.012359


In [ ]:
btc.to_csv("BTCUSDT_preprocessed.csv", index=False)
print("✔ BTCUSDT_preprocessed.csv saved successfully!")


✔ BTCUSDT_preprocessed.csv saved successfully!


In [ ]:
from google.colab import files
files.download("BTCUSDT_preprocessed.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
eth = pd.read_csv("/content/ETHUSDT_merged_hourly.csv")


In [ ]:
eth["open_time"] = pd.to_datetime(eth["open_time"])


In [ ]:
eth = eth.drop_duplicates(subset=["open_time"])


In [ ]:
eth = eth[[
    "open_time","open","high","low","close","volume",
    "quote_volume","trades","taker_buy_base","taker_buy_quote"
]]


In [ ]:
eth[[
    "open","high","low","close","volume",
    "quote_volume","trades","taker_buy_base","taker_buy_quote"
]] = eth[[
    "open","high","low","close","volume",
    "quote_volume","trades","taker_buy_base","taker_buy_quote"
]].astype(float)



In [ ]:
eth = eth.fillna(method="ffill")
eth.isnull().sum()

/tmp/ipython-input-1725314573.py:1: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  eth = eth.fillna(method="ffill")


,0
open_time,0
open,0
high,0
low,0
close,0
volume,0
quote_volume,0
trades,0
taker_buy_base,0
taker_buy_quote,0


In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

eth[[
    "open","high","low","close","volume",
    "quote_volume","trades","taker_buy_base","taker_buy_quote"
]] = scaler.fit_transform(
    eth[[
        "open","high","low","close","volume",
        "quote_volume","trades","taker_buy_base","taker_buy_quote"
    ]]
)
eth.head()

,open_time,open,high,low,close,volume,quote_volume,trades,taker_buy_base,taker_buy_quote
0,2022-01-01 00:00:00,0.871028,0.881174,0.878537,0.885741,0.019478,0.030909,0.020339,0.020005,0.034381
1,2022-01-01 01:00:00,0.885740,0.886977,0.890445,0.886322,0.019423,0.031041,0.020305,0.018265,0.031615
2,2022-01-01 02:00:00,0.886319,0.883838,0.893000,0.887400,0.010854,0.017343,0.013014,0.009774,0.016916
3,2022-01-01 03:00:00,0.887428,0.883376,0.892878,0.886030,0.007648,0.012215,0.010754,0.007722,0.013358
4,2022-01-01 04:00:00,0.886029,0.880567,0.888288,0.881081,0.013205,0.021018,0.013329,0.014537,0.025058


In [ ]:
eth.to_csv("ETHUSDT_preprocessed.csv", index=False)
print("✔ Saved ETHUSDT_preprocessed.csv")


✔ Saved ETHUSDT_preprocessed.csv


In [ ]:
from google.colab import files
files.download("ETHUSDT_preprocessed.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
bnb = pd.read_csv("/content/BNBUSDT_merged_hourly.csv")
bnb["open_time"] = pd.to_datetime(bnb["open_time"])
bnb = bnb.drop_duplicates(subset=["open_time"])

bnb = bnb[[
    "open_time","open","high","low","close","volume",
    "quote_volume","trades","taker_buy_base","taker_buy_quote"
]]

bnb[[
    "open","high","low","close","volume",
    "quote_volume","trades","taker_buy_base","taker_buy_quote"
]] = bnb[[
    "open","high","low","close","volume",
    "quote_volume","trades","taker_buy_base","taker_buy_quote"
]].astype(float)

bnb = bnb.fillna(method="ffill")

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
bnb[[
    "open","high","low","close","volume",
    "quote_volume","trades","taker_buy_base","taker_buy_quote"
]] = scaler.fit_transform(
    bnb[[
        "open","high","low","close","volume",
        "quote_volume","trades","taker_buy_base","taker_buy_quote"
    ]]
)
bnb.head()


/tmp/ipython-input-3752001411.py:18: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  bnb = bnb.fillna(method="ffill")


,open_time,open,high,low,close,volume,quote_volume,trades,taker_buy_base,taker_buy_quote
0,2022-01-01 00:00:00,0.542757,0.542538,0.553755,0.552407,0.031294,0.045365,0.039969,0.033211,0.045784
1,2022-01-01 01:00:00,0.552482,0.546184,0.561858,0.552239,0.027573,0.040228,0.040179,0.024134,0.033482
2,2022-01-01 02:00:00,0.552314,0.545024,0.563378,0.553245,0.014176,0.020681,0.024970,0.013599,0.018866
3,2022-01-01 03:00:00,0.553320,0.544195,0.563716,0.553748,0.013848,0.020196,0.027913,0.013731,0.019046
4,2022-01-01 04:00:00,0.553823,0.543698,0.559495,0.548549,0.018543,0.026965,0.028712,0.016232,0.022446


In [ ]:
bnb.to_csv("BNBUSDT_preprocessed.csv", index=False)
print("✔ Saved BNBUSDT_preprocessed.csv")


✔ Saved BNBUSDT_preprocessed.csv


In [ ]:
from google.colab import files
files.download("BNBUSDT_preprocessed.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
sol = pd.read_csv("/content/SOLUSDT_merged_hourly.csv")
sol["open_time"] = pd.to_datetime(sol["open_time"])
sol = sol.drop_duplicates(subset=["open_time"])

sol = sol[[
    "open_time","open","high","low","close","volume",
    "quote_volume","trades","taker_buy_base","taker_buy_quote"
]]

sol[[
    "open","high","low","close","volume",
    "quote_volume","trades","taker_buy_base","taker_buy_quote"
]] = sol[[
    "open","high","low","close","volume",
    "quote_volume","trades","taker_buy_base","taker_buy_quote"
]].astype(float)

sol = sol.fillna(method="ffill")

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
sol[[
    "open","high","low","close","volume",
    "quote_volume","trades","taker_buy_base","taker_buy_quote"
]] = scaler.fit_transform(
    sol[[
        "open","high","low","close","volume",
        "quote_volume","trades","taker_buy_base","taker_buy_quote"
    ]]
)

sol.head()


/tmp/ipython-input-3159908428.py:18: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  sol = sol.fillna(method="ffill")


,open_time,open,high,low,close,volume,quote_volume,trades,taker_buy_base,taker_buy_quote
0,2022-01-01 00:00:00,0.635381,0.641216,0.642452,0.644667,0.011231,0.016952,0.013496,0.011788,0.018095
1,2022-01-01 01:00:00,0.644653,0.647976,0.649752,0.646042,0.015453,0.023576,0.021087,0.014418,0.022384
2,2022-01-01 02:00:00,0.646067,0.643756,0.652847,0.646121,0.005180,0.007885,0.006711,0.005111,0.007914
3,2022-01-01 03:00:00,0.646185,0.643443,0.652648,0.646867,0.005781,0.008799,0.006156,0.006874,0.010642
4,2022-01-01 04:00:00,0.647010,0.643287,0.648641,0.641171,0.006667,0.010107,0.008015,0.006343,0.009781


In [ ]:
sol.to_csv("SOLUSDT_preprocessed.csv", index=False)
print("✔ Saved SOLUSDT_preprocessed.csv")

✔ Saved SOLUSDT_preprocessed.csv


In [ ]:
from google.colab import files
files.download("SOLUSDT_preprocessed.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
xrp = pd.read_csv("/content/XRPUSDT_merged_hourly.csv")
xrp["open_time"] = pd.to_datetime(xrp["open_time"])
xrp = xrp.drop_duplicates(subset=["open_time"])

xrp = xrp[[
    "open_time","open","high","low","close","volume",
    "quote_volume","trades","taker_buy_base","taker_buy_quote"
]]

xrp[[
    "open","high","low","close","volume",
    "quote_volume","trades","taker_buy_base","taker_buy_quote"
]] = xrp[[
    "open","high","low","close","volume",
    "quote_volume","trades","taker_buy_base","taker_buy_quote"
]].astype(float)

xrp = xrp.fillna(method="ffill")

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
xrp[[
    "open","high","low","close","volume",
    "quote_volume","trades","taker_buy_base","taker_buy_quote"
]] = scaler.fit_transform(
    xrp[[
        "open","high","low","close","volume",
        "quote_volume","trades","taker_buy_base","taker_buy_quote"
    ]]
)

xrp.head()

/tmp/ipython-input-1453277093.py:18: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  xrp = xrp.fillna(method="ffill")


,open_time,open,high,low,close,volume,quote_volume,trades,taker_buy_base,taker_buy_quote
0,2022-01-01 00:00:00,0.209610,0.207476,0.224433,0.213428,0.010797,0.006734,0.007981,0.009795,0.006050
1,2022-01-01 01:00:00,0.213444,0.212379,0.225385,0.213467,0.027907,0.017568,0.022068,0.027441,0.017116
2,2022-01-01 02:00:00,0.213444,0.209200,0.228363,0.214602,0.007855,0.004938,0.006079,0.009152,0.005697
3,2022-01-01 03:00:00,0.214618,0.209660,0.228860,0.214015,0.009174,0.005773,0.005559,0.008350,0.005205
4,2022-01-01 04:00:00,0.213992,0.208625,0.226460,0.211511,0.009415,0.005892,0.006353,0.007815,0.004842


In [ ]:
xrp.to_csv("XRPUSDT_preprocessed.csv", index=False)
print("✔ Saved XRPUSDT_preprocessed.csv")

✔ Saved XRPUSDT_preprocessed.csv


In [ ]:
from google.colab import files
files.download("XRPUSDT_preprocessed.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import pandas as pd

# -------------------------------------------
# STEP 1: LOAD THE PREPROCESSED DATA
# -------------------------------------------
# We load the preprocessed CSV created earlier.
# This file already has cleaned and normalized values.
btc = pd.read_csv("BTCUSDT_preprocessed.csv")

# Display first few rows to verify it loaded correctly
print("🔹 Loaded Dataset Preview:")
print(btc.head())


In [ ]:
#Train Test split

In [2]:
import pandas as pd

btc = pd.read_csv("/content/BTCUSDT_preprocessed.csv")

# Display first few rows to verify it loaded correctly
print(" Loaded Dataset Preview:")
print(btc.head())


 Loaded Dataset Preview:
             open_time      open      high       low     close    volume  \
0  2022-01-01 00:00:00  0.330079  0.334418  0.335309  0.334813  0.010957   
1  2022-01-01 01:00:00  0.334822  0.336779  0.339298  0.336130  0.006879   
2  2022-01-01 02:00:00  0.336139  0.336551  0.340912  0.336493  0.003536   
3  2022-01-01 03:00:00  0.336502  0.336419  0.341328  0.336509  0.004102   
4  2022-01-01 04:00:00  0.336518  0.336102  0.339486  0.335406  0.006282   

   quote_volume    trades  taker_buy_base  taker_buy_quote  
0      0.023250  0.026772        0.011748         0.024925  
1      0.014681  0.022101        0.007159         0.015279  
2      0.007559  0.016895        0.003008         0.006429  
3      0.008771  0.013787        0.003619         0.007738  
4      0.013399  0.016197        0.005794         0.012359  


In [3]:

# We convert 'open_time' from string format to datetime so that Python understands it as a time series.
btc["open_time"] = pd.to_datetime(btc["open_time"])

# Setting time as index ensures model processes data in order and enables time-based slicing.
btc.set_index("open_time", inplace=True)

print("\n DateTime Converted & Set as Index:")
print(btc.head())



 DateTime Converted & Set as Index:
                         open      high       low     close    volume  \
open_time                                                               
2022-01-01 00:00:00  0.330079  0.334418  0.335309  0.334813  0.010957   
2022-01-01 01:00:00  0.334822  0.336779  0.339298  0.336130  0.006879   
2022-01-01 02:00:00  0.336139  0.336551  0.340912  0.336493  0.003536   
2022-01-01 03:00:00  0.336502  0.336419  0.341328  0.336509  0.004102   
2022-01-01 04:00:00  0.336518  0.336102  0.339486  0.335406  0.006282   

                     quote_volume    trades  taker_buy_base  taker_buy_quote  
open_time                                                                     
2022-01-01 00:00:00      0.023250  0.026772        0.011748         0.024925  
2022-01-01 01:00:00      0.014681  0.022101        0.007159         0.015279  
2022-01-01 02:00:00      0.007559  0.016895        0.003008         0.006429  
2022-01-01 03:00:00      0.008771  0.013787        0.003

In [4]:
# Time-series data must NOT be randomized.
# We split based on row counts, so older data becomes training
# and recent data remains for testing/future prediction.
train_size = int(len(btc) * 0.8)

train = btc.iloc[:train_size]   # First 80% rows
test = btc.iloc[train_size:]    # Remaining 20% rows

print("\n Split Completed!")
print("Train Size:", len(train))
print("Test Size:", len(test))



 Split Completed!
Train Size: 21042
Test Size: 5261


In [6]:
# We want to predict future "close" price, so it is the target (y).
# All other columns are features (X).
train_X = train.drop("close", axis=1)
train_y = train["close"]

test_X = test.drop("close", axis=1)
test_y = test["close"]

print("\n Columns in Feature Set:")
print(train_X.columns)
print("\n Target Column: close")



 Columns in Feature Set:
Index(['open', 'high', 'low', 'volume', 'quote_volume', 'trades',
       'taker_buy_base', 'taker_buy_quote'],
      dtype='object')

 Target Column: close


In [7]:

print("\n DATA READY FOR LSTM SEQUENCE CREATION")
print("Training Feature Shape:", train_X.shape)
print("Training Target Shape:", train_y.shape)
print("Testing Feature Shape:", test_X.shape)
print("Testing Target Shape:", test_y.shape)



 DATA READY FOR LSTM SEQUENCE CREATION
Training Feature Shape: (21042, 8)
Training Target Shape: (21042,)
Testing Feature Shape: (5261, 8)
Testing Target Shape: (5261,)


In [10]:
print("printing the data after spliting:")
btc.head()

printing the data after spliting:


,open,high,low,close,volume,quote_volume,trades,taker_buy_base,taker_buy_quote
open_time,,,,,,,,,
2022-01-01 00:00:00,0.330079,0.334418,0.335309,0.334813,0.010957,0.023250,0.026772,0.011748,0.024925
2022-01-01 01:00:00,0.334822,0.336779,0.339298,0.336130,0.006879,0.014681,0.022101,0.007159,0.015279
2022-01-01 02:00:00,0.336139,0.336551,0.340912,0.336493,0.003536,0.007559,0.016895,0.003008,0.006429
2022-01-01 03:00:00,0.336502,0.336419,0.341328,0.336509,0.004102,0.008771,0.013787,0.003619,0.007738
2022-01-01 04:00:00,0.336518,0.336102,0.339486,0.335406,0.006282,0.013399,0.016197,0.005794,0.012359
